# Tensorflow MNIST Classifier demo

>⚠️ **Warning:** Some of the attacks in this demo, _deepfool_ and _CW_ in particular, are computationally expensive and will take a very long to complete if run using the CPUs found in a typical personal computer.
> For this reason, it is highly recommended that you run these demos on a CUDA-compatible GPU.

This notebook contains an end-to-end demostration of Dioptra that can be run on any modern laptop.
Please see the [example README](README.md) for instructions on how to prepare your environment for running this example.

## Setup

Below we import the necessary Python modules and ensure the proper environment variables are set so that all the code blocks will work as expected,

In [ ]:
# Import packages from the Python standard library
import importlib.util
import os
import sys
import pprint
import time
import warnings
from pathlib import Path


def register_python_source_file(module_name: str, filepath: Path) -> None:
    """Import a source file directly.

    Args:
        module_name: The module name to associate with the imported source file.
        filepath: The path to the source file.

    Notes:
        Adapted from the following implementation in the Python documentation:
        https://docs.python.org/3/library/importlib.html#importing-a-source-file-directly
    """
    spec = importlib.util.spec_from_file_location(module_name, str(filepath))
    module = importlib.util.module_from_spec(spec)
    sys.modules[module_name] = module
    spec.loader.exec_module(module)


# Filter out warning messages
warnings.filterwarnings("ignore")

# Experiment name
EXPERIMENT_NAME = "mnist_feature_squeezing"

# Default address for accessing the RESTful API service
RESTAPI_ADDRESS = "http://localhost:80"

# Set DIOPTRA_RESTAPI_URI variable if not defined, used to connect to RESTful API service
if os.getenv("DIOPTRA_RESTAPI_URI") is None:
    os.environ["DIOPTRA_RESTAPI_URI"] = RESTAPI_ADDRESS

# Default address for accessing the MLFlow Tracking server
MLFLOW_TRACKING_URI = "http://localhost:35000"

# Set MLFLOW_TRACKING_URI variable, used to connect to MLFlow Tracking service
if os.getenv("MLFLOW_TRACKING_URI") is None:
    os.environ["MLFLOW_TRACKING_URI"] = MLFLOW_TRACKING_URI

# Path to workflows archive
WORKFLOWS_TAR_GZ = Path("workflows.tar.gz")

# Register the examples/scripts directory as a Python module
register_python_source_file("scripts", Path("..", "scripts", "__init__.py"))

from scripts.client import DioptraClient
from scripts.utils import make_tar

# Import third-party Python packages
import numpy as np
from mlflow.tracking import MlflowClient

# Create random number generator
rng = np.random.default_rng(54399264723942495723666216079516778448)

In [ ]:
mnist_model = "feature_squeezing_mnist_le_net" 
mnist_shallow = "feature_squeezing_mnist_shallow_net"
model_id = 1
model_id_shallow = 1
mnist_dataset_training = "/dioptra/data/Mnist/training"
mnist_dataset_testing = "/dioptra/data/Mnist/testing"
mlflow_queue = "tensorflow_cpu"

In [ ]:
def mlflow_run_id_is_not_known(response):
    return response["mlflowRunId"] is None and response["status"] not in [
        "failed",
        "finished",
    ]

def print_response(jobtype, jobname, response):
    print(f"{jobtype} for job {jobname} submitted.")
    print("")
    pprint.pprint(response)
    print("")
    
def wait_for_job(response):
    while mlflow_run_id_is_not_known(response):
        time.sleep(1)
        response = restapi_client.get_job_by_id(response["jobId"]) 
    return response

## Dataset

We obtained a copy of the MNIST dataset when we ran `download_data.py` script. If you have not done so already, see [How to Obtain Common Datasets](https://pages.nist.gov/dioptra/getting-started/acquiring-datasets.html).
The training and testing images for the MNIST dataset are stored within the `/dioptra/data/Mnist` directory as PNG files that are organized into the following folder structure,

    Mnist
    ├── testing
    │   ├── 0
    │   ├── 1
    │   ├── 2
    │   ├── 3
    │   ├── 4
    │   ├── 5
    │   ├── 6
    │   ├── 7
    │   ├── 8
    │   └── 9
    └── training
        ├── 0
        ├── 1
        ├── 2
        ├── 3
        ├── 4
        ├── 5
        ├── 6
        ├── 7
        ├── 8
        └── 9

The subfolders under `training/` and `testing/` are the classification labels for the images in the dataset.
This folder structure is a standardized way to encode the label information and many libraries can make use of it, including the Tensorflow library that we are using for this particular demo.

## Submit and run jobs

The entrypoints that we will be running in this example are implemented in the Python source files under `src/` and the `src/MLproject` file.
To run these entrypoints within Dioptra's architecture, we need to package those files up into an archive and submit it to the Dioptra RESTful API to create a new job.
For convenience, we provide the `make_tar` helper function defined in `examples/scripts/utils.py`.

In [ ]:
make_tar(["src"], WORKFLOWS_TAR_GZ)

To connect with the endpoint, we will use a client class defined in the `examples/scripts/client.py` file that is able to connect with the Dioptra RESTful API using the HTTP protocol.
We connect using the client below.
The client uses the environment variable `DIOPTRA_RESTAPI_URI`, which we configured at the top of the notebook, to figure out how to connect to the Dioptra RESTful API.

In [ ]:
restapi_client = DioptraClient()

We need to register an experiment under which to collect our job runs.
The code below checks if the relevant experiment named `"mnist"` exists.
If it does, then it just returns info about the experiment, if it doesn't, it then registers the new experiment.

In [ ]:
response_experiment = restapi_client.get_experiment_by_name(name=EXPERIMENT_NAME)

if response_experiment is None or "Not Found" in response_experiment.get("message", []):
    response_experiment = restapi_client.register_experiment(name=EXPERIMENT_NAME)

response_experiment

Next, we need to train our model.
Depending on the specs of your computer, training either the shallow net model or the LeNet-5 model on a CPU can take 10-20 minutes or longer to complete.
If you are fortunate enough to have access to a dedicated GPU, then the training time will be much shorter.

So that we do not start this code by accident, we are embedding the code in a text block instead of keeping it in an executable code block.
**If you need to train one of the models, create a new code block and copy and paste the code into it.**

In [ ]:
response_shallow_train = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="train",
    entry_point_kwargs=" ".join(
        [
            "-P model_architecture=shallow_net",
            "-P epochs=30",
            f"-P register_model_name={mnist_shallow}",
            f"-P training_dir={mnist_dataset_training}",
        ]
    ),
    queue=mlflow_queue,
)

print_response("Training", "shallow neural network", response_shallow_train)

response_le_net_train = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="train",
    entry_point_kwargs=" ".join(
        [
            "-P model_architecture=le_net",
            "-P epochs=30",
            f"-P register_model_name={mnist_model}",
            f"-P training_dir={mnist_dataset_training}",
        ]
    ),
    queue=mlflow_queue,
)

print_response("Training", "LeNet-5 neural network", response_le_net_train)

The following code block generates adversarial images on the MNIST dataset using the Fast Gradient Method attack and then attempts to classify the adversarial images.

| parameter | type | description |
| --- | --- | --- |
| `eps` | _float_ | Attack step size. \[default: 0.3\] |
| `eps_step` | _float_ | Step size of input variation for minimal perturbation computation. [default: 0.1] |
| `targeted` | _bool_ | Indicates whether the attack is targeted (True) or untargeted (False). [default: False] |
| `num_random_init` | _int_ | Number of random initializations within the epsilon ball. For ``random_init=0`` starting at the original input. [default: 0] |
| `minimal` | _bool_ | Indicates if computing the minimal perturbation (True). If True, also define eps_step for the step size and eps for the maximum perturbation. [default: False] |

In [ ]:
response_fgm_le_net = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="fgm",
    entry_point_kwargs=" ".join(
        [
            f"-P model_name={mnist_model}",
            f"-P data_dir={mnist_dataset_testing}",
            f"-P model_version={model_id}"
        ]
    ),
    queue = mlflow_queue,
    timeout="1h"

)

print_response("Attack", "FGM", response_fgm_le_net)
response_fgm_le_net = wait_for_job(response_fgm_le_net)
    
response_infer = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="infer",
    entry_point_kwargs=" ".join(
        [
            f"-P run_id={response_fgm_le_net['mlflowRunId']}",
            f"-P model_version={model_id}",
            f"-P model_name={mnist_model}",
        ]
    ),
    queue = mlflow_queue,
    depends_on=response_fgm_le_net["jobId"]
)
    
print("Dependent jobs submitted")

This block does the same as the previous block, but applies the feature squeezing defense between the attack and infer steps.
This pre-processing defense compresses the images being classified by our neural network such that their color depth is reduced to a binary, monochrome pallete.
The level of compression can be tuned by adjusting the bit_depth parameter below (use values between 1 (binary) and 8 (original image color depth) to tune the defense.

**FGM parameters**

| parameter | type | description |
| --- | --- | --- |
| `eps` | _float_ | Attack step size. \[default: 0.3\] |
| `eps_step` | _float_ | Step size of input variation for minimal perturbation computation. [default: 0.1] |
| `targeted` | _bool_ | Indicates whether the attack is targeted (True) or untargeted (False). [default: False] |
| `num_random_init` | _int_ | Number of random initializations within the epsilon ball. For ``random_init=0`` starting at the original input. [default: 0] |
| `minimal` | _bool_ | Indicates if computing the minimal perturbation (True). If True, also define eps_step for the step size and eps for the maximum perturbation. [default: False] |

**Feature squeezing parameters**

| parameter | type | description |
| --- | --- | --- |
| `bit_depth` | _int_ | An integer between 1-8 that defines the color depth of the squeezed image. [default: 8] |

In [ ]:
response_fgm_le_net = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="fgm",
    entry_point_kwargs=" ".join(
        [
            f"-P model_name={mnist_model}",
            f"-P data_dir={mnist_dataset_testing}",
            f"-P model_version={model_id}"
        ]

    ),
    queue = mlflow_queue,

)

print_response("Attack", "FGM", response_fgm_le_net)
response_fgm_le_net = wait_for_job(response_fgm_le_net)

response_feature_squeeze = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="feature_squeeze",
    entry_point_kwargs=" ".join(
        [
            f"-P run_id={response_fgm_le_net['mlflowRunId']}",
            f"-P model={mnist_model}/{model_id}",
            "-P model_architecture=le_net",
            "-P bit_depth=1",
            "-P batch_size=32"
        ]
    ),
    queue = mlflow_queue,
    depends_on=response_fgm_le_net["jobId"],
    timeout = "1h"
)

print_response("Defense", "Feature Squeezing", response_feature_squeeze)
response_feature_squeeze = wait_for_job(response_feature_squeeze)

response_infer_defended = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="infer",
    entry_point_kwargs=" ".join(
        [
            f"-P run_id={response_feature_squeeze['mlflowRunId']}",
            f"-P model_version={model_id}",
            f"-P model_name={mnist_model}",
        ]
    ),
    queue = mlflow_queue,
    depends_on=response_feature_squeeze["jobId"]
)
    
print("Dependent jobs submitted")

This block uses the carlini-wagner attack with the Linf distance metric to generate adversarial images and checks the model's accuracy against the attack.

**Carlini Wagner Parameters**

| parameter | type | description |
| --- | --- | --- |
| `targeted` | _bool_ | Indicates whether the attack is targeted (True) or untargeted (False). [default: False] |
| `learning_rate` | _float_ | The initial learning rate for the attack algorithm. Smaller values produce better results but are slower to converge. [default: 0.01] |

In [ ]:
response_cw_le_net = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="cw_inf",
    entry_point_kwargs=" ".join(
        [
            f"-P model_name={mnist_shallow}",
            f"-P model_version={model_id_shallow}",
            "-P model_architecture=shallow_net",
            "-P targeted=True",
            "-P max_iter=10",
            "-P confidence=0.0",
            f"-P testing_dir={mnist_dataset_testing}",
            "-P learning_rate=0.01",
            "-P verbose=True",
            "-P batch_size=32"
        ]

    ),
    queue = mlflow_queue,

)

print_response("Attack", "CW Inf", response_cw_le_net)
response_cw_le_net = wait_for_job(response_cw_le_net)

response_le_net_infer_le_net_fgm = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="infer",
    entry_point_kwargs=" ".join(
         [
             f"-P run_id={response_cw_le_net['mlflowRunId']}",
             f"-P model_version={model_id_shallow}",
             f"-P model_name={mnist_shallow}",
        ]
    ),
    queue = mlflow_queue,
    depends_on=response_cw_le_net["jobId"],
)

print("Dependent jobs submitted")

This block does the same as the previous block, but applies the feature squeezing defense between the attack and infer steps.

In [ ]:
response_cw_le_net = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="cw_inf",
    entry_point_kwargs=" ".join(
        [
            f"-P model_name={mnist_shallow}", 
            f"-P model_version={model_id_shallow}", 
            "-P model_architecture=shallow_net",
            "-P targeted=True", 
            "-P max_iter=20",
            "-P confidence=0.0", 
            f"-P testing_dir={mnist_dataset_testing}", 
            "-P learning_rate=0.01", 
            "-P verbose=True", 
            "-P batch_size=32"
        ]

    ),
    queue = mlflow_queue,
)

print_response("Attack", "CW Inf", response_cw_le_net)
response_cw_le_net = wait_for_job(response_cw_le_net)

response_feature_squeeze = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="feature_squeeze",
    entry_point_kwargs=" ".join(
        [
            f"-P run_id={response_cw_le_net['mlflowRunId']}",
            f"-P model={mnist_shallow}/{model_id_shallow}",
            "-P model_architecture=shallow_net",
            "-P bit_depth=1",
            "-P batch_size=32"
        ]
    ),
    depends_on=response_cw_le_net["jobId"],
    queue = mlflow_queue,
)

print_response("Defense", "Feature Squeezing", response_feature_squeeze)
response_feature_squeeze = wait_for_job(response_feature_squeeze)

response_le_net_infer_le_net_cw = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="infer",
    entry_point_kwargs=" ".join(
        [
            f"-P run_id={response_feature_squeeze['mlflowRunId']}",
            f"-P model_name={mnist_shallow}",
            f"-P model_version={model_id_shallow}"
        ]
    ),
    depends_on=response_feature_squeeze["jobId"],
    queue = mlflow_queue,
)

print("Dependent jobs submitted")

This block uses the carlini-wagner attack using the L2 distance metric to generate adversarial images and checks the model's accuracy against the attack.

In [ ]:
response_cw_shallow_net = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="cw_l2",
    entry_point_kwargs=" ".join(
        [
            f"-P model_name={mnist_shallow}",
            f"-P model_version={model_id_shallow}",
            "-P binary_search_steps=50", 
            "-P initial_const=0.01",
            "-P model_architecture=shallow_net",
            "-P max_iter=10",
            f"-P testing_dir={mnist_dataset_testing}",
            "-P verbose=True",
            "-P batch_size=1"]

    ),
    queue = mlflow_queue,

)

print_response("Attack", "CW L2", response_cw_shallow_net)
response_cw_shallow_net = wait_for_job(response_cw_shallow_net)

response_le_net_infer_shallow_net_cw = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="infer",
    entry_point_kwargs=" ".join(
        [
            f"-P run_id={response_cw_shallow_net['mlflowRunId']}",
            f"-P model_name={mnist_model}",
            f"-P model_version={model_id}",
        ]
    ),
    queue = mlflow_queue,
    depends_on=response_cw_shallow_net["jobId"],
)

print("Dependent jobs submitted")


This block does the same as the previous block, but applies the feature squeezing defense between the attack and infer steps.

In [149]:
response_cw_shallow_net = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="cw_l2",
    entry_point_kwargs=" ".join(
        [
            f"-P model_name={mnist_shallow}", 
            f"-P model_version={model_id_shallow}", 
            "-P binary_search_steps=50", 
            "-P initial_const=0.01",
            "-P model_architecture=shallow_net",
            "-P max_iter=10",
            f"-P testing_dir={mnist_dataset_testing}", 
            "-P verbose=True", 
            "-P batch_size=32"
        ]

    ),
    queue = mlflow_queue,
)

print_response("Attack", "CW L2", response_cw_shallow_net)
response_cw_shallow_net = wait_for_job(response_cw_shallow_net)

response_feature_squeeze = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="feature_squeeze",
    entry_point_kwargs=" ".join(
        [
            f"-P run_id={response_cw_shallow_net['mlflowRunId']}",
            f"-P model={mnist_shallow}/{model_id_shallow}",
            "-P model_architecture=shallow_net",
            "-P bit_depth=1",
            "-P batch_size=32"
        ]
    ),
    depends_on=response_cw_le_net["jobId"],
    queue = mlflow_queue,
)

print_response("Defense", "Feature Squeezing", response_feature_squeeze)
response_feature_squeeze = wait_for_job(response_feature_squeeze)

response_le_net_infer_le_net_cw = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="infer",
    entry_point_kwargs=" ".join(
        [
            f"-P run_id={response_feature_squeeze['mlflowRunId']}",
            f"-P model_name={mnist_shallow}",
            f"-P model_version={model_id_shallow}"
        ]
    ),
    depends_on=response_feature_squeeze["jobId"],
    queue = mlflow_queue,
)

print("Dependent jobs submitted")

Dependent jobs submitted


This block uses the Deepfool attack to generate adversarial MNIST images, applies the feature squeezing defense, and checks the model's accuracy against the defended adversarial dataset.

**Unique Deepfool parameters**

| parameter | type | description |
| --- | --- | --- |
| `epsilon` | _float_ | Overshoot parameter. [default: 0.00001] |
| `nb_grads` | _int_ | Number of class gradients to compute. [default: 10] |

In [ ]:
response_deepfool_le_net = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="deepfool",
    entry_point_kwargs=" ".join(
        [
            f"-P model_name={mnist_model}",
            f"-P training_dir={mnist_dataset_training}",
            "-P batch_size=1",
            "-P max_iter=10",
            "-P nb_grads=10",
            "-P epsilon=0.001",
            "-P image_size=[28,28]"
        ],
    ),
    queue = mlflow_queue,
)

print_response("Attack", "Deepfool", response_deepfool_le_net)
response_deepfool_le_net = wait_for_job(response_deepfool_le_net)

response_le_net_infer_deepfool = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="infer",
    entry_point_kwargs=" ".join(
        [
            f"-P run_id={response_deepfool_le_net['mlflowRunId']}",
            f"-P model_name={mnist_model}",
        ]
    ),
    depends_on=response_deepfool_le_net["jobId"],
    queue = mlflow_queue,
)

print("Dependent jobs submitted")

This block does the same as the previous block, but applies the feature squeezing defense between the attack and infer steps.

In [ ]:
response_deepfool_le_net = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="deepfool",
    entry_point_kwargs=" ".join(
        [
            f"-P model_name={mnist_model}",
            f"-P training_dir={mnist_dataset_training}",
            "-P batch_size=1",
            "-P max_iter=10",
            "-P nb_grads=10",
            "-P epsilon=0.01",
        ],
    ),
    queue = mlflow_queue,
)

print_response("Attack", "Deepfool", response_deepfool_le_net)
response_deepfool_le_net = wait_for_job(response_deepfool_le_net)

response_feature_squeeze = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="feature_squeeze",
    entry_point_kwargs=" ".join(
        [
            f"-P run_id={response_deepfool_le_net['mlflowRunId']}",
            f"-P model={mnist_model}/{model_id}",
            "-P model_architecture=le_net",
            "-P bit_depth=1",
            "-P batch_size=1",
        ],
    ),
    depends_on=response_deepfool_le_net["jobId"],
    queue = mlflow_queue,
)
print_response("Defense", "Feature Squeezing", response_feature_squeeze)
response_feature_squeeze = wait_for_job(response_feature_squeeze)

response_le_net_infer_le_net_fgm = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="infer",
    entry_point_kwargs=" ".join(
        [
            f"-P run_id={response_feature_squeeze['mlflowRunId']}",
            f"-P model_name={mnist_model}",
        ],
    ),
    depends_on=response_feature_squeeze["jobId"],
    queue = mlflow_queue,
)

print("Dependent jobs submitted")

This block applies the Jacobian Saliency Map Approach attack to generate adversarial images for the MNIST dataset.

**Unique JSMA parameters**

| parameter | type | description |
| --- | --- | --- |
| `theta` | _float_ | Amount of Perturbation introduced to each modified feature per step (can be positive or negative). [default: 0.1] |
| `gamma` | _float_ | Maximum fraction of features being perturbed (between 0 and 1). [default: 1.0] |

In [ ]:
response_jsma = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="jsma",
    entry_point_kwargs=" ".join(
        [
            f"-P model_name={mnist_model}",
            f"-P model_version={model_id}", 
            "-P theta=4.5",
            "-P gamma=1.0", 
            f"-P testing_dir={mnist_dataset_testing}"
        ] 

    ),
    queue = mlflow_queue,

)

print_response("Attack", "JSMA", response_jsma)
response_jsma = wait_for_job(response_jsma)

response_le_net_infer_jsma = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="infer",
    entry_point_kwargs=" ".join(
        [
            f"-P run_id={response_jsma['mlflowRunId']}",
            f"-P model_name={mnist_model}",
            f"-P model_version={model_id}",
        ]
    ),
    queue = mlflow_queue,
    depends_on=response_jsma["jobId"],
)

print("Dependent jobs submitted")

This block does the same as the previous block, but applies the feature squeezing defense between the attack and infer steps.

In [ ]:
response_jsma = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="jsma",
    entry_point_kwargs=" ".join(
        [
            f"-P model_name={mnist_model}",
            f"-P model_version={model_id}", 
            "-P theta=4.5",
            "-P gamma=1.0", 
            f"-P testing_dir={mnist_dataset_testing}"
        ] 
    ),
    queue = mlflow_queue,
)

print_response("Attack", "JSMA", response_jsma)
response_jsma = wait_for_job(response_jsma)

response_feature_squeeze = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="feature_squeeze",
    entry_point_kwargs=" ".join(
        [
            f"-P run_id={response_jsma['mlflowRunId']}",
            f"-P model={mnist_model}/{model_id}",
            "-P model_architecture=le_net",
            "-P bit_depth=1",
            "-P batch_size=32"
        ]
    ),
    queue = mlflow_queue,
    depends_on=response_jsma["jobId"],
)

print_response("Defense", "Feature Squeezing", response_feature_squeeze)
response_feature_squeeze = wait_for_job(response_feature_squeeze)

response_le_net_infer_le_net_jsma = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="infer",
    entry_point_kwargs=" ".join(
        [
            f"-P run_id={response_feature_squeeze['mlflowRunId']}",
            f"-P model_name={mnist_model}",
            f"-P model_version={model_id}",
        ]
    ),
    queue = mlflow_queue,
    depends_on=response_feature_squeeze["jobId"],
)

print("Dependent jobs submitted")